In [1]:
import os
import sys

import numpy as np
import torch

sys.path.append("..")

from models.score_ligandmpnn import LigandMPNNBatch, score_complex

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(
    "../models/model_params/ligandmpnn_v_32_020_25.pt",
    map_location=device,
    weights_only=True,
)
ligandmpnn = LigandMPNNBatch(
    ligand_mpnn_use_side_chain_context=True,
    device=device,
)
ligandmpnn.load_state_dict(checkpoint["model_state_dict"])
ligandmpnn.eval().to(device)

LigandMPNNBatch(
  (features): ProteinFeaturesLigand(
    (embeddings): PositionalEncodings(
      (linear): Linear(in_features=66, out_features=16, bias=True)
    )
    (edge_embedding): Linear(in_features=416, out_features=128, bias=False)
    (norm_edges): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (node_project_down): Linear(in_features=148, out_features=128, bias=True)
    (norm_nodes): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (type_linear): Linear(in_features=147, out_features=64, bias=True)
    (y_nodes): Linear(in_features=147, out_features=128, bias=False)
    (y_edges): Linear(in_features=16, out_features=128, bias=False)
    (norm_y_edges): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (norm_y_nodes): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (W_v): Linear(in_features=128, out_features=128, bias=True)
  (W_c): Linear(in_features=128, out_features=128, bias=True)
  (W_nodes_y): Linear(in_features=128, out_features=

In [2]:
pdb_path = "../pdbs/NbALFA_ALFAtag.pdb"
chains_to_design = "A"
redesigned_residues = "A1 A3 A4 A5 A7 A8 A9 A13 A14 A15 A19 A20 A21 A23 A24 A25 A26 A27 A39 A41 A44 A45 A46 A48 A50 A52 A53 A67 A68 A69 A72 A73 A74 A75 A76 A77 A78 A79 A80 A81 A82 A83 A84 A85 A86 A88 A89 A91 A92 A93 A95 A97 A99 A100 A102 A114 A116 A118 A119 A120 A121 A123 A124"
seqs_list = [
    "SGEVQLQESGGGLVQPGGSLRLSCTASGVTISALNAMAMGWYRQAPGERRVMVAAVSERGNAMYRESVQGRFTVTRDFTNKMVSLQMDNLKPEDTAVYYCHVLEDRVDSFHDYWGQGTQVTVSS:PSRLEEELRRRLTEP",
    "GGTVVLTESGGGTVAPGGSATLTATASGVTISALNAMAWGWYRQRPGERPVAVAAVSERGNAMYREDVRGRWTVTADRANKTVSLEMRDLQPEDTATYYPHVLEDRVDSFHDYWGAGVPLTVVP:PSRLEEELRRRLTEP",
    "GQVQLQQSAELARPGASVKMSCKASGYTFTSQAPGKGLEWVSAITWNELARPGASVKMSGHIDYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKYPYYGSHWYFDVWGAGTTVTVS:PSRLEEELRRRLTEP",
    "PGLRAEDTAVYYCAKYPYELARPGYTFTSQAPGKGLGSHWYFDVWWYFDLYQMNSLRATIRDNSKNTWVSEVWGAGTASKMSCKASGGSVKMEDTAVYYCAKYPYYGSHGAGTDNSKNAVVTVS:PSRLEEELRRRLTEP",
]

entropy, loss, perplexity = score_complex(
    ligandmpnn,
    pdb_path,
    # chains_to_design=chains_to_design,
    redesigned_residues=redesigned_residues,
    # seqs_list=seqs_list,
    use_side_chain_context=True,
)
entropy.shape, loss, perplexity

(torch.Size([1, 134, 20]),
 tensor([[1.1179, 3.0119, 0.2660, 2.5777, 0.2208, 1.0683, 0.0761, 0.0767, 0.0759,
          2.3542, 0.0976, 3.3412, 0.0988, 0.0936, 0.0999, 0.5448, 1.6589, 2.7531,
          0.1709, 1.1868, 0.1492, 1.8021, 0.1264, 1.0390, 0.8123, 2.3950, 0.8369,
          2.4463, 2.7029, 2.7629, 1.4452, 4.6227, 3.3798, 3.1338, 0.5154, 2.0458,
          0.0831, 0.1128, 5.2675, 2.6589, 0.2862, 0.6491, 0.8641, 0.1051, 1.1638,
          4.5884, 1.1727, 2.0618, 1.7215, 0.8974, 0.3654, 0.2081, 0.2496, 1.1256,
          2.2644, 5.0136, 0.1530, 4.3272, 1.1144, 0.4414, 0.1351, 3.7346, 1.4417,
          2.2810, 0.6260, 2.7551, 0.1076, 0.6286, 1.4993, 0.7472, 0.9102, 0.7306,
          1.6116, 0.4630, 1.6891, 2.7592, 1.0569, 0.2140, 4.2956, 0.6524, 0.3296,
          0.1119, 2.2601, 1.1815, 2.6056, 2.2750, 0.3840, 3.1506, 1.1450, 0.8629,
          0.0869, 0.1910, 0.1079, 1.5957, 0.1146, 0.5283, 0.4175, 5.7748, 1.5257,
          1.3712, 3.4429, 3.1166, 3.3000, 2.6661, 1.7946, 0.7124, 1.678

In [3]:
from models.score_ligandmpnn import extract_from_sample, extract_from_score

In [4]:
# !sh "./score_complex.sh" "../pdbs/NbALFA_ALFAtag_AF3.pdb" "../results_/score/"
# extract_from_score("../results_/score/NbALFA_ALFAtag_AF3.pt")

In [5]:
# !sh "./score_wt.sh" "../pdbs/NbALFA_ALFAtag_AF3.pdb" "../results/score"
# extract_from_score("../results/score/NbALFA_ALFAtag_AF3.pt")

In [6]:
# !sh "./sample_complex.sh" "../pdbs/NbALFA_AF3.pdb" "../results/sample"
# extract_from_sample("../results/sample/stats/NbALFA_AF3.pt")